In [1]:
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
import pandas as pd
import string 
import re

2025-08-23 11:45:26.806729: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755949526.963767      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755949527.019817      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [9]:
df = pd.read_csv('/kaggle/input/flickr8k/captions.txt')

In [10]:
df = df.groupby('image').head(2).reset_index(drop=True)
df.head(10)

,image,caption
0,1000268201_693b08cb0e.jpg,A child in a pink dress is climbing up a set o...
1,1000268201_693b08cb0e.jpg,A girl going into a wooden building .
2,1001773457_577c3a7d70.jpg,A black dog and a spotted dog are fighting
3,1001773457_577c3a7d70.jpg,A black dog and a tri-colored dog playing with...
4,1002674143_1b742ab4b8.jpg,A little girl covered in paint sits in front o...
5,1002674143_1b742ab4b8.jpg,A little girl is sitting in front of a large p...
6,1003163366_44323f5815.jpg,A man lays on a bench while his dog sits by him .
7,1003163366_44323f5815.jpg,A man lays on the bench to which a white dog i...
8,1007129816_e794419615.jpg,A man in an orange hat starring at something .
9,1007129816_e794419615.jpg,A man wears an orange hat and glasses .


In [11]:
df = df.drop(['image'],axis=1)

In [12]:
def infonce_loss(zi,zj):
    zi = tf.nn.l2_normalize(zi)
    zj = tf.nn.l2_normalize(zj)
    logits = tf.matmul(zi,(tf.transpose(zj)))
    logits = tf.nn.softmax(logits)
    labels = tf.eye(tf.shape(logits)[-1])
    zi_zj_loss = tf.keras.losses.CategoricalCrossentropy(labels,logits)
    zj_zi_loss = tf.keras.losses.CategoricalCrossentropy(labels,tf.transpose(logits))
    total_loss = zi_zj_loss + zj_zi_loss
    return total_loss

16182

In [ ]:
class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, input_dim, output_dim, mask_zero = True, **kwargs):
        super().__init__(**kwargs)
        self.token_embeddings = layers.Embedding(
            input_dim=input_dim, output_dim=output_dim,mask_zero=mask_zero)
        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=output_dim,mask_zero=False)
        self.sequence_length = sequence_length
        self.input_dim = input_dim
        self.output_dim = output_dim

    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions

class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads

        self.attention = layers.MultiHeadAttention(num_heads=num_heads, key_dim=int(embed_dim/num_heads))
        self.dense_proj = keras.Sequential([
            layers.Dense(dense_dim, activation="relu"),
            layers.Dense(embed_dim)
        ])
        self.layernorm_1 = layers.LayerNormalization(epsilon=1e-5)
        self.layernorm_2 = layers.LayerNormalization(epsilon=1e-5)
        self.dropout_1 = layers.Dropout(0.1)

    def call(self, inputs, mask=None):
        # Convert mask to boolean with shape (batch, 1, seq_len)
        if mask is not None:
            mask = tf.cast(mask[:, tf.newaxis, :], dtype=tf.bool)

        attention_output = self.attention(
            query=inputs,
            value=inputs,
            key=inputs,
            attention_mask=mask
        )
        attention_output = self.dropout_1(attention_output)
        proj_input = self.layernorm_1(inputs + attention_output)

        proj_output = self.dense_proj(proj_input)
        return self.layernorm_2(proj_input + proj_output)


class TransformerDecoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads

        self.attention_1 = layers.MultiHeadAttention(num_heads=num_heads, key_dim=int(embed_dim/num_heads))
        self.attention_2 = layers.MultiHeadAttention(num_heads=num_heads, key_dim=int(embed_dim/num_heads))
        self.dense_proj = keras.Sequential([
            layers.Dense(dense_dim, activation="relu"),
            layers.Dense(embed_dim)
        ])
        self.dropout_1 = layers.Dropout(0.1)
        self.dropout_2 = layers.Dropout(0.1)
        self.layernorm_1 = layers.LayerNormalization(epsilon=1e-5)
        self.layernorm_2 = layers.LayerNormalization(epsilon=1e-5)
        self.layernorm_3 = layers.LayerNormalization(epsilon=1e-5)

    # def get_causal_attention_mask(self, inputs):
    #     seq_len = tf.shape(inputs)[1]
    #     causal_mask = tf.linalg.band_part(tf.ones((seq_len, seq_len), dtype=tf.bool), -1, 0)
    #     return causal_mask[tf.newaxis, :, :]  # (1, seq_len, seq_len)

    def call(self, inputs, encoder_outputs, mask=None):
        # Padding mask: (batch_size, 1, seq_len)
        if mask is not None:
            padding_mask = tf.cast(mask[:, tf.newaxis, :], dtype=tf.bool)
        else:
            padding_mask = None

        # # Causal mask: (1, seq_len, seq_len)
        # causal_mask = self.get_causal_attention_mask(inputs)

        # # Combine masks for self-attention
        # if padding_mask is not None:
        #     combined_mask = tf.logical_and(padding_mask, causal_mask)
        # else:
        #     combined_mask = causal_mask

        # Self-attention with combined mask
        attention_output_1 = self.attention_1(
            query=inputs,
            value=inputs,
            key=inputs,
            attention_mask=None,
            use_causal_mask=True
        )
        attention_output_1 = self.dropout_1(attention_output_1)
        attention_output_1 = self.layernorm_1(inputs + attention_output_1)

        # Cross-attention with padding mask only
        attention_output_2 = self.attention_2(
            query=attention_output_1,
            value=encoder_outputs,
            key=encoder_outputs,
            attention_mask=padding_mask,
            use_causal_mask=False
        )
        attention_output_2 = self.dropout_2(attention_output_2)
        attention_output_2 = self.layernorm_2(attention_output_1 + attention_output_2)

        proj_output = self.dense_proj(attention_output_2)
        return self.layernorm_3(attention_output_2 + proj_output)

In [ ]:
with strategy.scope():
    embed_dim = 256
    dense_dim = 1024
    num_heads = 8
    num_blocks = 10
    
    encoder_inputs = tf.keras.Input(shape=(None,), dtype="int32", name="encoder_inputs")
    decoder_inputs = tf.keras.Input(shape=(None,), dtype="int32", name="decoder_inputs")
    
    # Padding masks
    encoder_mask = tf.keras.layers.Lambda(lambda x: tf.cast(tf.not_equal(x, 0), tf.bool))(encoder_inputs)
    # cross_attention_mask = tf.keras.layers.Lambda(lambda x: tf.cast(x[:, tf.newaxis, tf.newaxis, :], tf.bool))(encoder_mask) 
    
    # Embeddings
    encoder_embed = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(encoder_inputs)
    decoder_embed = PositionalEmbedding(256, 257, embed_dim,mask_zero=False)(decoder_inputs)
    
    # Encoder blocks
    x = encoder_embed
    for _ in range(num_blocks):
        x = TransformerEncoder(embed_dim, dense_dim, num_heads)(x, mask=encoder_mask)
    encoder_outputs = x
    
    # Decoder blocks
    x = decoder_embed
    for _ in range(num_blocks):
        x = TransformerDecoder(embed_dim, dense_dim, num_heads)(x, encoder_outputs, mask=encoder_mask)
    
    # Final layers
    x = layers.LayerNormalization(epsilon=1e-5)(x)
    x = layers.Dropout(0.1)(x)
    decoder_outputs = layers.Dense(256)(x)

    transformer = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [ ]:
def train_step(xi, xj):
    with tf.GradientTape() as tape:
        zi = transformer(xi, training=True)
        zj = transformer(xj, training=True)

        loss = infonce_loss(zi,zj)
        gradients = tape.gradient(loss, transformer.trainable_variables)
        optimizer.apply_gradients(zip(gradients,transformer.trainable_variables))    
        return loss    